In [1]:
# import statements for rest of notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
#plt.rc('text', usetex=True)
plt.rc('font', family='serif')
mpl.rcParams.update({'font.size': 20})
mpl.rcParams['figure.figsize'] = 8,8
import math as m   # importing math library outside function definition saves time 
import numpy as np
from IPython.display import Image
import scipy as sci
from scipy.misc import derivative
import scipy.optimize as opt
import sympy as sym
from numpy import pi
from itertools import chain, repeat
import warnings
warnings.filterwarnings("ignore")#turn off all warnings
from mpmath import mp
import mpmath
mp.prec = 100


Things to know about this notebook:
The goal of this project is to provide a toolbox that students or teachers could use to demonstrate and explore concepts in geometrical optics. Ray tracing is the method of choice to achieve these objectives. The simulator is written such that it can be extended and tinkered with as the user sees fit. 

We have taken an object oriented approach to programming this simulator as it provides clairty into how the various parts interact. Throughout the notebook we will comment and annotate code to explain the more complicated segments and illustrate how all the pieces come together. 

Useful uncategorized information  

Coordinate System:
    Currently the lenses are restricted to the first quandrant of the XY plane. Most examples are set up with the intended 
    direction of propagation in the +y direction. If we refer to things being below or above others, that is to say that the y
    coordinate of the below item is less than the above object. 

Lenses:
    Lenses are represented as a list[] of ordered pairs (domain, surfaces). Each ordered pair contains
    a domain list and a surface list. The domain list defines a piecewise definition of the segment of the lens. 
    If multiple surfaces are present over one domain, they are given in no particular order. For clarity imagine the following:
             -------f(x)-------
    from x0  -------g(x)------- to x1 would be translated as [[x0,x1],[g,f]] where this is one element of the lens
    
    The lens (domain, surfaces) elements are ordered by the domain component. The domain for a given lens must be continuous. 
    Hence if we have the element [[x0,x1],[f1,g1]] then the next element should be [[x1,x2],[f2,g2]] where x2 > x1 and the 
    fi, gi's are surfaces defined over the given domains. 
    
    Vertical lines require special attention. Many other curves in the xy plane may be represented as functions of x 
    but vertical lines cannot be. Hence we elected to give vertical lines a domain of x = c and the 
    surfaces are the y points that define the vertical line segment. This translates to the following:
    
    y1
    |
    |
    |
    |
    |
    |
    |
    |
    y0 at x = x0 -> [[x0],[y0,y1]]
    
    The lens class has predefined lenses in it along with the capability to do gaussian optics as specified by the user.
    
Rays:
    Rays carry a lot of information in them. They are defined as a geometrical ray. With a source location and an angle measured
    counter-clockwise from the horizonal (in radians). 
    
RayGenerator:
    This makes the initial list of rays that will propagate through the system. It can create point sources, (parallel) beam 
    sources, and converging (beam) sources. 
    
Driver:
    This class binds all of the constitutent parts together. It is responsible for handling ray propagation, ray reflection/
    refraction, and plotting the results.

 

Our approach:

    Surface Finding:
        Each ray is modeled as a function of x (unless it's vertical) and each part of the lens is also. Hence we can use a
        numerical root solver in the scipy optimize library to determine where the ray will cross the surface, if ever. We can 
        repeat this for every surface avaliable and take the surface which minimizes the distance from our orginial point as
        this minimizes the optical path length and would therefore be the one the ray would actually follow. 
        
        This code will not work for media whose index of refraction is position dependent.
        
    Refracting:
        We employ snell's law as our method of refracting rays. To determine the normal vector we leverage the functional form 
        of our surfaces and calculate the normal vector based on differential calculus. Once we have that, we use the dot 
        product to calculate the angle of incidence and snell's law for the angle of transmission. Finally, to avoid the
        complications induced from having several coordinate systems, we calculate the xy components of the refracted/reflected
        ray by using the 2D rotation matrix for a vector. 


In [2]:
'''
The ray class is esentially a container for a variety of attributes associated with a light ray

We have broken down which category each property best fits into
this typically will indicate which part of the program will use the field although
there may be overlap with other categories.

Summary of fields:
The light ray is defined in a mathematical sense as a normal ray which has an end point (source location) and an
angle that provides the ray direction (denoted angle) measured CCW from the horizontal. 

Due to our method of determining where a ray will strike a surface. 
''' 

class Ray:
    rayCount = 0
    def __init__(self, x, y, angle, wavelength=0,identity=42, intensity=1, objIndex=-1, sPolarization=0.5, pPolarization=0.5):
        
        #vector quantities
        self.sourceLocation = (x,y)
        self.angle = angle
        self.unitVector = [np.cos(angle), np.sin(angle)]
        
        #line quantities
        self.slope = (self.unitVector[1]/self.unitVector[0])
        self.yIntercept = y - x*self.slope # y = mx + b --> b = y - mx
        self.xIntercept = - self.yIntercept/self.slope # x = -b/m
        
        #plot quantities
        self.locationHistory = [self.sourceLocation] # keep track of the prevous interface points
        self.angleHistory = [angle*180/np.pi]
        
        #light quantities
        self.wavelength = wavelength
        self.intensity = intensity
        self.intensityList = [intensity]
        self.parallel = pPolarization
        self.perpendicular = sPolarization
        
        #program quantities
        self.objIndex = -1
        #self.identity = identity #this can be set up in the ray generator
        self.identity = Ray.rayCount
        Ray.rayCount+=1
        self.reflected = 0
        self.normalVectorHistory = []
        self.worthUsing = 1
    
    
    def update_location_history(self, x, y, angle, objIndex):
        
        #update vector quantities:
        self.sourceLocation = (x,y)
        self.angle = angle
        
        ang = mpmath.mpmathify(angle)
        x0,y0 = mpmath.cos(ang),mpmath.sin(ang)
        self.unitVector = [float(x0),float(y0)]
        
        self.slope = float(mpmath.fdiv(y0,x0))
        self.yIntercept = y - x*self.slope # y = mx + b --> b = y - mx
        self.xIntercept = - self.yIntercept/self.slope # x = -b/m
        
        #update plot quantities
        self.locationHistory.append((x,y))
        self.angleHistory.append(angle*180/np.pi)
        
        #update program quantity
        self.objIndex = objIndex
        
        
    def rPerp(angleI, angleT, nI, nT, wavelength):
        return (nI(wavelength)*np.cos(angleI)-nT(wavelength)*np.cos(angleT))/(nI(wavelength)*np.cos(angleI)+nT(wavelength)*np.cos(angleT))
    def rPara(angleI, angleT, nI, nT, wavelength):
        return (nT(wavelength)*np.cos(angleI)-nI(wavelength)*np.cos(angleT))/(nT(wavelength)*np.cos(angleI)+nI(wavelength)*np.cos(angleT))
    def tPerp(angleI, angleT, nI, nT, wavelength):
        return (2*nI(wavelength)*np.cos(angleI))/(nI(wavelength)*np.cos(angleI)+nT(wavelength)*np.cos(angleT))
    def tPara(angleI,angleT, nI, nT, wavelength):
        return (2*nI(wavelength)*np.cos(angleI))/(nT(wavelength)*np.cos(angleI)+nI(wavelength)*np.cos(angleT))
    
    #should be irradience
    def update_intensity_history(self, angleI, angleT, nI, nT):
        wavelength = self.wavelength
        if(angleI == 0):  
            self.intensity *= (self.perpendicular + self.parallel)*nT(wavelength)/nI(wavelength)*(2*nI(wavelength)/(nI(wavelength)+nT(wavelength)))**2
            self.intensityList.append(self.intensity)
        else:
            R = self.parallel*Ray.rPara(angleI,angleT,nI,nT,wavelength)**2 + self.perpendicular*Ray.rPerp(angleI,angleT,nI, nT, wavelength)**2
            T = (nT(self.wavelength)*np.cos(angleT)/(nI(self.wavelength)*np.cos(angleI)))*(self.parallel*Ray.tPara(angleI,angleT, nI,nT,wavelength)**2 + self.perpendicular*Ray.tPerp(angleI,angleT, nI,nT, wavelength)**2)
            if(angleI == angleT):
                self.intensity*=R
                self.intensityList.append(self.intensity)
            else:
                self.intensity*=T
                self.intensityList.append(self.intensity)
                
    def printString(self):
        return "{0}\n{1}\n{2}\n".format(self.slope, self.xIntercept,np.asarray(self.angleHistory)*180/np.pi)

In [9]:
class RayGenerator:
    """
    This class acts as a container for a ray list to be given to a driver class along some optical elements
    for those rays to interact with.
    
    There are three predefined methods of construction that provide frequently seen ray sources. Those 
    are explained in turn."""
    def __init__(self, rayList):
        self.theRayList = rayList
        
    @classmethod
    def point_source(cls, x0, y0, numberOfRays, startAngle, stopAngle, startLambda=1, endLambda=1, sPolarization=0.5, pPolarization=0.5):
        """
        Overview :
        This sets up a diverging source eminating from a point (x0,y0) sweeping from start angle to stop angle in evenly spaced
        intervals based on how many rays are given.

        Inputs:
        x0,y0 - (x,y) coordinates of the source
        startAngle, stopAngle - angles to begin and end the generation process (in radians)
        startLambda, endLambda - the wavelengths to begin and end at for the rays(they're divided in the same way the angles are)
        sPolarization, pPolarization - the fraction of perpendicular or parallel light, these must add to 1

        Outputs:
        a ray list which fits these criteria
        """
        if startAngle == 0 and stopAngle == 2*np.pi:
            theAngleList = np.linspace(startAngle, stopAngle, numberOfRays+1)
        else:
            theAngleList = np.linspace(startAngle, stopAngle, numberOfRays)
        rayList = []
        for i, angle in enumerate(theAngleList):#still have to put in the color code
            if(angle < 0):
                angle+=2*pi
            rayList.append(Ray(x0,y0,angle, 0, sPolarization=sPolarization, pPolarization=pPolarization))
        return cls(rayList)
    
    @classmethod
    def beam_source(cls, x0, y0, x1, y1, numberOfRays, startLambda=1, endLambda=1, sPolarization=0.5, pPolarization=0.5):
        """
        Overview - this creates a line segment determined by points (x0,y0) and (x1,y1) that the rays will emerge perpendicularly 
        from at evenly space intervals based on the number of rays. If only one ray is given, the ray will emminate from the 
        (x0,y0) point. Since this simulator was originally based around having light move from top to bottom, hence the light always is oriented
        to move upward.

        Inputs:
        x0,y0 - start point of the line segment
        x1,y1 - end point of the line segment
        numberOfRays - self evident
        OTHER PARAMETERS - explained above

        Outputs:
        a list of rays which satify these conditions

        """
        if x1 - x0 == 0 and y1 - y0 == 0 :
            raise(Exception("beam width must be non-zero"))
        rayAngle = np.arctan2((x1-x0),-(y1-y0))
        xList, yList = np.linspace(x0, x1, numberOfRays), np.linspace(y0, y1, numberOfRays)
        pointList = list(zip(xList, yList))
        rayList = []
        for i, point in enumerate(pointList):
            rayList.append(Ray(point[0],point[1],rayAngle,0,identity=i, sPolarization=sPolarization, pPolarization=pPolarization))
        return cls(rayList)
    
    @classmethod
    def converging_source(cls, x0, y0, x1, y1, numberOfRays, startAngle, stopAngle, startLambda=1, endLambda=1, sPolarization=0.5, pPolarization=0.5):
        """
        Overview - this constructor seeks to combine the beam source with the point source to create a converging source from a line.
        Inputs:
        x0,y0 - define the start of the line
        x1,y1 - define the end of the line
        startAngle - angle of the leftmost ray
        stopAngle - angle of the rightmost ray (with rays inbetween having angles that evenly divide the range)
        OTHER PARAMETERS - see point source constructor

        Outputs:
        a ray list that meets these specifications

        """
        if x1-x0 == 0 and y1-y0 == 0:
            raise(Exception("converging beam width must be non-zero"))
        xList, yList = np.linspace(x0, x1, numberOfRays), np.linspace(y0, y1, numberOfRays)
        pointList = list(zip(xList, yList))
        theAngleList = np.linspace(startAngle, stopAngle, numberOfRays)
        rayList = []
        for i, point in enumerate(pointList):
            rayList.append(Ray(point[0], point[1], theAngleList[i], 0, sPolarization=sPolarization, pPolarization=pPolarization))
        return cls(rayList)
        

In [10]:
class Mirror:
    def __init__(self, surfaceList = [], lossFunction=lambda i : 0):
        self.surfaceList = surfaceList
        self.lossFunction = lossFunction
    
    @classmethod
    def predefined(cls, name, lossFunction=lambda i : 0, xOffSet=0, yOffSet=0):
        """
        Inputs:
        name - name of the desired mirror
        lossFunction - a function to reduce the intensity of incident light #### NOT IMPLEMENTED YET 
        xOffSet, yOffSet - horizontal and vertical displacements

        Outputs:
        A mirror with the specified shape

        Description:
        This constructor provides several example mirrors that could be used on their own or studied to
        learn how to build another mirror
        """
        if(name == "concave up"):
            f = lambda x : -(-(x-xOffSet)**2 + 9)**(1/2) + 5 + yOffSet
            return cls(surfaceList =[[[xOffSet, 3+xOffSet],[f]]],lossFunction=lossFunction)
        elif(name == "concave down"):
            f = lambda x : (-(x-xOffSet)**2 + 16)**(1/2) + 5 + yOffSet
            return cls(surfaceList =[[[xOffSet, 4+xOffSet],[f]]],lossFunction=lossFunction)
        elif(name == "horizontal"):
            f = lambda x : 4 + yOffSet
            return cls(surfaceList = [[[xOffSet, 5+xOffSet], [f]]], lossFunction=lossFunction)
        elif(name == "vertical"):
            return cls(surfaceList = [[[3+xOffSet], [1+yOffSet, 6+yOffSet]]],lossFunction=lossFunction)
        elif(name == "parabolic down"):
            return cls(surfaceList =[[[xOffSet,10+xOffSet],[lambda x : -(x-5-xOffSet)**2+8+yOffSet]]])
        elif(name == "hyperbolic down"):
            return cls(surfaceList = [[[xOffSet,10+xOffSet],[lambda x: -((x-5-xOffSet)**2+1)**(1/2)+8+yOffSet]]])
        else:
            raise(Exception("Unknown predefined case\nThe acceptable cases are: concave up, concave down, vertical, and horizontal"))
    @classmethod
    def gaussianOptics(cls, R=0, vertical=0, diameter=0, xOffSet=0, yOffSet=0, lossFunction = lambda i : 0):
        """
        Inputs:
        R - radius of curvature of the mirror, negative indicates that the center is below the arc of the mirror
        vertical - if R = 0 then we have a planar mirror, vertical is a flag to indicate the orientation of the mirror
        diameter - specifies the horizontal extent of the mirror (or vertical extent if R = 0 and vertical = 1)
        xOffSet, yOffSet - self explanatory
        lossFunction - a function to describe how much energy is lost upon a reflection ### not implemented

        Outputs:
        An instance of mirror with the traits required by the parameters.

        Description:
        Specifies a mirror by the radius of curvature, if the radius is negavtive, that indicates that the center is below 
        the arc of the mirror (or that the center has a lesser y coordinate compared to the arc)

        """
        if(diameter or R):
            if(R > 0):
                f = lambda x : -(-(x-xOffSet)**2 + R*R)**(1/2) + yOffSet
                if(diameter):
                    if(diameter < 2*R):
                        return cls(surfaceList=[[[xOffSet-diameter/2, xOffSet+diameter/2],[f]]],lossFunction=lossFunction)
                    else:
                        raise(Exception("This diameter exceeds the domain of the mirror surface"))
                return cls([[[xOffSet-R, xOffSet+R],[f]]],lossFunction =lossFunction)
            elif(R < 0):
                R *= -1
                f = lambda x : (-(x-xOffSet)**2 + R**2)**(1/2) + yOffSet
                if(diameter):
                    if(diameter < 2*R):
                        return cls(surfaceList=[[[xOffSet-diameter/2, xOffSet+diameter/2],[f]]],lossFunction=lossFunction)
                    else:
                        raise(Exception("This diameter exceeds the domain of the mirror surface"))
                return cls(surfaceList=[[[xOffSet-R, xOffSet+R],[f]]],lossFunction=lossFunction)
            elif(R == 0 and diameter):
                if(vertical):
                    return cls(surfaceList=[[[xOffSet],[yOffSet-diameter/2, yOffSet+diameter/2]]])
                return cls(surfaceList = [[[xOffSet-diameter/2,xOffSet+diameter/2], [lambda x : 3+yOffSet]]])
        else:
            raise(Exception("The diameter needs to be specified for a planar mirror"))
    @classmethod
    def custom(cls,surfaceList,lossFunction=lambda i:0):
        return cls(surfaceList=surfaceList,lossFunction=lossFunction)     

In [11]:
class Lens:
    def __init__(self,surfaceList=[],n=lambda x : 1.5):
        
        self.refractiveIndex = n
        self.surfaceList = surfaceList

    @classmethod
    def predefined(cls, name, nFunction=lambda x : 1.5, xOffSet=0, yOffSet=0):#I should probably include the r1, r2, height, and width values in these examples
        """
        ***NOTE***
        Conventions: 
        1. the rays typically move vertically from - to +
        When we say one thing is before another we mean that the former has a lesser "y" value than the latter
        
        Inputs:
        name - the type of predefined lens desired
        nFunction - an index of refraction function based on wavelength
        xOffSet, yOffSet - horizontal and vertical displacements
        
        Output:
        An instance of the class that the specified lens
        
        Description:
        The primary purpose of this construtor is to illustrate the formate of building a lens
        The portions that aren't labeled hyperbolic or elliptic follow the conventions of gaussian optics which are
        detailed in that constructor
        
        """
        #should probably change the convex parts so that they reflect the updated methodology
        if(name == "biconvex"):
            f1 = lambda x : -(-(x-5-xOffSet)**2 + 9)**(1/2) + 5 + yOffSet
            f2 = lambda x : (-(x-5-xOffSet)**2 + 9)**(1/2) + 1 +yOffSet
            x0 = opt.fsolve(lambda x: f1(x)-f2(x), 2+xOffSet, xtol=1e-10, maxfev=200)
            x1 = opt.fsolve(lambda x: f1(x)-f2(x), 7+xOffSet, xtol=1e-10, maxfev=200)
            return cls(surfaceList = [[[x0[0],x1[0]],[f1,f2]]],n=nFunction)
        elif(name == "biconcave"):
            mySurfaceList = []
            f1 = lambda x : (-(x-5-xOffSet)**2 + 9)**(1/2) + 1 +yOffSet
            f2 = lambda x : -(-(x-5-xOffSet)**2 + 9)**(1/2) + 8 + yOffSet
            mySurfaceList.append([[3+xOffSet],[f1(3+xOffSet),f2(3+xOffSet)]])
            mySurfaceList.append([[3+xOffSet,7+xOffSet],[f1,f2]])
            mySurfaceList.append([[7+xOffSet],[f1(7+xOffSet),f2(7+xOffSet)]])
            return cls(surfaceList=mySurfaceList,n=nFunction)  
        elif(name == "planar convex"):
            f1 = lambda x : -(-(x-5-xOffSet)**2 + 9)**(1/2) + 5 + yOffSet
            f2 = lambda x : 3 + yOffSet
            x0 = opt.fsolve(lambda x: f1(x)-f2(x), 2+xOffSet, xtol=1e-10, maxfev=200)
            x1 = opt.fsolve(lambda x: f1(x)-f2(x), 7+xOffSet, xtol=1e-10, maxfev=200)
            return cls(surfaceList = [[[x0[0],x1[0]],[f1,f2]]],n=nFunction)
        elif(name == "planar concave"):
            mySurfaceList = []
            f1 = lambda x : (-(x-5-xOffSet)**2 + 9)**(1/2) + 1 +yOffSet
            f2 = lambda x : 6 + yOffSet
            mySurfaceList.append([[3+xOffSet],[f1(3+xOffSet),f2(3+xOffSet)]])
            mySurfaceList.append([[3+xOffSet,7+xOffSet],[f1,f2]])
            mySurfaceList.append([[7+xOffSet],[f1(7+xOffSet),f2(7+xOffSet)]])
            return cls(surfaceList=mySurfaceList, n=nFunction)
        elif(name == "meniscus convex"):
            f1 = lambda x: -(-(x-5-xOffSet)**2 + 16)**(1/2) + 6+yOffSet
            f2 = lambda x : -(-(x-5-xOffSet)**2 + 36)**(1/2) + 9+yOffSet
            x0 = opt.fsolve(lambda x: f1(x)-f2(x), 2+xOffSet, xtol=1e-10, maxfev=200)
            x1 = opt.fsolve(lambda x: f1(x)-f2(x), 8+xOffSet, xtol=1e-10, maxfev=200)
            return cls(surfaceList = [[[x0[0],x1[0]],[f1,f2]]],n=nFunction)
        elif(name == "meniscus concave"):#I could save using the append opperation and memory initialization by just expliciting spelling out what I wanted my list to be
            mySurfaceList = []
            f1 = lambda x : -(-(x-5-xOffSet)**2 + 9)**(1/2) + 4.5+ yOffSet
            f2 = lambda x : -(-(x-5-xOffSet)**2 + 49)**(1/2) + 8 + yOffSet
            mySurfaceList.append([[3+xOffSet],[f1(3+xOffSet),f2(3 + xOffSet)]])
            mySurfaceList.append([[3+xOffSet,7+xOffSet],[f1,f2]])
            mySurfaceList.append([[7+xOffSet],[f1(7+xOffSet),f2(7 + xOffSet)]])
            return cls(surfaceList = mySurfaceList,n=nFunction)
        elif(name == "hyberbolic biconvex"):
            f1 = lambda x : -(-5/9*(x-5-xOffSet)**2+5)**(1/2) + yOffSet
            f2 = lambda x : (-5/9*(x-5-xOffSet)**2+5)**(1/2) + yOffSet
            return cls(surfaceList=[[[2+xOffSet],[f2(2+xOffSet), f1(2+xOffSet)]],[[2+xOffSet,8+xOffSet],[f2,f1]],[[8+xOffSet],[f2(8+xOffSet),f1(8+xOffSet)]]],n = nFunction)
        elif(name == "hyberbolic plano convex"):
            f1 = lambda x : -((x-5-xOffSet)**2+1)**(1/2)+8 + yOffSet
            f2 = lambda x : 1+yOffSet
            return cls(surfaceList=[[[3+xOffSet],[f2(3+xOffSet), f1(3+xOffSet)]],[[3+xOffSet,7+xOffSet],[f2,f1]],[[7+xOffSet],[f2(7+xOffSet),f1(7+xOffSet)]]], n = nFunction)
        elif(name == "ellipse biconvex"):
            f1 = lambda x : -(-5/9*(x-5-xOffSet)**2+5)**(1/2) +yOffSet
            f2 = lambda x : (-5/9*(x-5-xOffSet)**2+5)**(1/2) + yOffSet
            return cls(surfaceList=[[[2+xOffSet],[f2(2+xOffSet), f1(2+xOffSet)]],[[2+xOffSet,8+xOffSet],[f2,f1]],[[8+xOffSet],[f2(8+xOffSet),f1(8+xOffSet)]]],n = nFunction)
        else:
            raise(Exception("Unknown lens type, please enter one of the following:\nbiconvex, biconcave, planar convex, planar concave, meniscus convex, meniscus concave"))
    
    
    @classmethod
    def gaussianOptics(cls,r1=0,r2=0,thickness=0,diameter=0,xOffSet=0,yOffSet=0,nFunction=lambda x:1.5):
        """
        ***NOTE***
        Conventions: 
        1. the rays typically move vertically from - to +
        When we say one thing is before another we mean that the former has a lesser "y" value than the latter
        
        2. if the arc that defines a surface of a lens comes before the center, that radius is positive, and negative otherwise
        
        3. in place of using infinity for describing planar surfaces, we let the radius go to zero because this is an otherwise unused
        value
        
        Inputs:
        r1 - radius of curvature of the first surface (in the vertical direction)
        r2 - radius of curvature of the second surface
        thickness - the vertical distance between the vertices of the two surfaces
        diameter - the length of the domain of the lens (their horizontal extent)
        xOffSet, yOffSet - horiztonal and vertical displacements respectively
        nFunction - a index of refraction function dependent on wavelength
        
        Output:
        An instance on the lens class with a surface built by the inputs. 
        
        ***NOTE*** 
        Due to some quirk in the program, the lens doesn't always behave correctly when it is centered on the y axis,
        therefore an offset is built in to ensure that this isn't normally encountered
        
        Description:
        Using the conventions of gaussian optics and our surface representation scheme we build the
        lens specified by the parameters given. There are six lenses that may be produced. Those are as follows:
        biconvex, biconcave, meniscus convex, meniscus concave, plano concave, plano convex. 
        
        Below are some helper functions to condense the code and prevent having to rewrite the same code.
        
        After those are a variety of cases for producing different lenses based on the inputs.
        
        """
        
        def abrvFunc(r1,r2,f1,f2,diameter,x0=0,x1=0,xOffSet=0, thickness=0): #this is a helper function to make the code more readable(or less eye watering if you prefer)
            rmax = max(r1,r2)
            if(x0 != 0 or x1 != 0):
                if(abs(x1[0]-x0[0]) <= diameter):#we can either raise an exception or just use the maximal diameter
                    raise(Exception("this diameter exceeds the limitations imposed by the specified diameter"))
            xmin = rmax - diameter/2
            xmax = rmax + diameter/2
            mySurfaceList = [[[xmin+xOffSet],sorted([f1(xmin+xOffSet),f2(xmax+xOffSet)])]]
            mySurfaceList.append([[xmin+xOffSet,xmax+xOffSet],[f1,f2]])
            mySurfaceList.append([[xmax+xOffSet],sorted([f1(xmax+xOffSet),f2(xmax+xOffSet)])])
            return mySurfaceList
        
        def widthHandler(f1, f2, x0, x1, surfaceList):
            f1y0, f1y1, f2y0, f2y1 = f1(x0), f1(x1), f2(x0), f2(x1)
            if(f1y0 < f2y0 - 0.001 or f1y0 > f2y0 + 0.001):
                surfaceList.insert(0,[[x0],sorted([f1y0, f2y0])])
            if(f1y1 < f2y1 - 0.001 or f1y1 > f2y1 + 0.001):
                surfaceList.append([[x1],sorted([f1y1, f2y1])])
        
        if(((r1 or r2 or diameter) and thickness) or ((r1 or r2) and diameter) or (r1 and r2)):#this should say that at least two of the following(r1,r2,width,height) 
            #have to be given 
            
            if(thickness < 0 or diameter < 0): #deal with negative width
                raise(Exception("width and height are constrained to be greater than or equal to zero"))
            
            elif(thickness == 0 and diameter == 0 ):
                raise(Exception("at minimum a width or height must be specified for a lens along with r1 or r2"))
            
            elif(r1 > 0 and r2 > 0 and r1 < r2):#meniscus convex case 1 #fixed
                if(not thickness):
                    thickness = abs(r1-(r1**2 - diameter**2/4)**(1/2) - r2 + (r2**2-diameter**2/4)**(1/2))
                rmax = r2
                f1 = lambda x : -(-(x-rmax-xOffSet)**2 + r1**2)**(1/2) + rmax+1 +yOffSet -r2 + r1 - thickness
                f2 = lambda x : -(-(x-rmax-xOffSet)**2 + r2**2)**(1/2) + rmax+1 +yOffSet
                d = -r1 + r2 + thickness
                if(d < 0):
                    d = -d
                elif(d == 0):
                    d = 0.001
                a = (1/d)*(4*d**2*r1**2 - (d**2 - r2**2 + r1**2)**2)**(1/2)
                x = (d**2 - r2**2 + r1**2)/(2*abs(d))
                x0 = [-a/2 +rmax + xOffSet]
                x1 = [a/2 + rmax + xOffSet]
                if(diameter):
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,x0,x1,xOffSet=xOffSet),n=nFunction)
                surfaceList = [[[x0[0],x1[0]],[f1,f2]]]
                widthHandler(f1,f2,x0[0],x1[0],surfaceList)
                return cls(surfaceList = surfaceList,n=nFunction)
                
            elif(r1 < 0 and r2 < 0 and r2 > r1): #meniscus convex case 2 #fixed I think
                if(not thickness):
                    thickness = abs(r1-(r1**2 - diameter**2/4)**(1/2) - r2 + (r2**2-diameter**2/4)**(1/2))
                r1, r2 = -r1,-r2
                rmax = r1
                f1 = lambda x : (-(x-rmax-xOffSet)**2 + r1**2)**(1/2) + 1 +yOffSet 
                f2 = lambda x : (-(x-rmax-xOffSet)**2 + r2**2)**(1/2) +1 +yOffSet + r1 + thickness - r2
                d = r1 - r2 + thickness
                if(d < 0):
                    d = -d
                elif(d == 0):
                    d = 0.001
                a = (1/d)*(4*d**2*r1**2 - (d**2 - r2**2 + r1**2)**2)**(1/2)
                x = (d**2 - r2**2 + r1**2)/(2*abs(d))
                x0 = [-a/2 +rmax + xOffSet]
                x1 = [a/2 + rmax + xOffSet]
                if(diameter):
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,x0,x1,xOffSet=xOffSet),n=nFunction)
                surfaceList = [[[x0[0],x1[0]],[f1,f2]]]
                widthHandler(f1,f2,x0[0],x1[0],surfaceList)
                return cls(surfaceList = surfaceList,n=nFunction)
                
            elif(r1 > 0 and r2 == 0):#plano-convex case 1 #fixed
                if(not thickness):
                    thickness = abs(r1-(r1**2-diameter**2/4)**(1/2))
                rmax = r1
                f1 = lambda x : -(-(x-rmax-xOffSet)**2 + r1**2)**(1/2) + 1 +yOffSet+rmax
                f2 = lambda x : 1+yOffSet + thickness
                d = rmax - thickness
                b = -2*(rmax+xOffSet)
                c = d**2 + (rmax+xOffSet)**2 - rmax**2
                x0 = [(-b-(b**2 - 4*c)**(1/2))/2]
                x1 = [(-b+(b**2 - 4*c)**(1/2))/2]
                
                if(diameter):    
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,x0,x1,xOffSet=xOffSet),n=nFunction)
                surfaceList = [[[x0[0],x1[0]],[f1,f2]]]
                widthHandler(f1,f2,x0[0],x1[0],surfaceList)
                return cls(surfaceList = surfaceList,n=nFunction)
            
            elif(r1 > 0 and r2 < 0):#biconvex #fixed
                r2 = -r2
                if(not thickness):
                    thickness = abs(r1-(r1**2-diameter**2/4)**(1/2)) + abs(r2-(r2**2-diameter**2/4)**(1/2))
                rmax = max(r1, r2)
                rmin = min(r1,r2)
                f1 = lambda x : -(-(x-rmax-xOffSet)**2 + r1**2)**(1/2) + rmax+1+yOffSet
                f2 = lambda x : (-(x-rmax-xOffSet)**2 + r2**2)**(1/2) + rmax+1 + thickness-r1-r2+yOffSet
                d = r1 + r2 - thickness
                if(d < 0):
                    d = -d
                elif(d == 0):
                    d = 0.001
                a = (1/d)*(4*d**2*r1**2 - (d**2 - r2**2 + r1**2)**2)**(1/2)
                x = (d**2 - r2**2 + r1**2)/(2*abs(d))
                x0 = [-a/2 +rmax + xOffSet]
                x1 = [a/2 + rmax + xOffSet]
                if(diameter):
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,x0,x1,xOffSet=xOffSet),n=nFunction)
                surfaceList = [[[x0[0],x1[0]],[f1,f2]]]
                widthHandler(f1,f2,x0[0],x1[0],surfaceList)
                return cls(surfaceList = surfaceList,n=nFunction)

            elif(r1 == 0 and r2 > 0):#plano-concave case 1
                if(diameter == 0 or thickness == 0):
                    raise(Exception("Insufficient information provided for concave class object, both diameter and thickness must be postive"))
                rmax = r2
                f1 = lambda x : 1+yOffSet-thickness
                f2 = lambda x : -(-(x-rmax-xOffSet)**2 + r2**2)**(1/2) + 1+rmax+yOffSet
                return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,xOffSet=xOffSet),n=nFunction) 

            elif(r1 == 0 and r2 == 0):#rectangle
                mySurfaceList = [[[1+xOffSet],[1+yOffSet,thickness+1+yOffSet]]]
                mySurfaceList.append([[1+xOffSet,diameter+1+xOffSet],[lambda x : 1+yOffSet, lambda x : thickness+1+yOffSet]])
                mySurfaceList.append([[diameter+1+xOffSet],[1+yOffSet,thickness+1+yOffSet]])
                return cls(surfaceList=mySurfaceList,n=nFunction)

            elif(r1 == 0 and r2 < 0):#plano-convex case 2
                if(not thickness):
                    thickness = abs(r2-(r2**2-diameter**2/4)**(1/2))
                r2 = -r2
                rmax = r2
                f1 = lambda x : (-(x-rmax-xOffSet)**2 + r2**2)**(1/2) +1+yOffSet 
                f2 = lambda x : 1+yOffSet+r2-thickness
                d = rmax - thickness
                b = -2*(rmax+xOffSet)
                c = d**2 + (rmax+xOffSet)**2 - rmax**2
                x0 = [(-b-(b**2 - 4*c)**(1/2))/2]
                x1 = [(-b+(b**2 - 4*c)**(1/2))/2]
                if(diameter):
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,x0,x1,xOffSet=xOffSet),n=nFunction)
                surfaceList = [[[x0[0],x1[0]],[f1,f2]]]
                widthHandler(f1,f2,x0[0],x1[0],surfaceList)
                return cls(surfaceList = surfaceList,n=nFunction)
                
            elif(r1 < 0 and r2 > 0):#biconcave
                if(diameter == 0 or thickness == 0):
                    raise(Exception("Insufficient information provided for concave class object, both diameter and thickness must be positive"))
                r1 = -r1
                rmax = max(r1,r2)
                f1 = lambda x : (-(x-rmax-xOffSet)**2 + r1**2)**(1/2) + rmax+1+yOffSet
                f2 = lambda x : -(-(x-rmax-xOffSet)**2 + r2**2)**(1/2) + rmax+1 + thickness +r1+r2+yOffSet
                return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,xOffSet=xOffSet),n=nFunction)

            elif(r1 < 0 and r2 == 0):#plano-concave case 2
                if(diameter == 0 or thickness == 0):
                    raise(Exception("Insufficient information provided for concave class object, both diameter and thickness must be positive"))
                r1 = -r1
                rmax = r1
                f1 = lambda x : (-(x-rmax-xOffSet)**2 + r1**2)**(1/2) + 1+yOffSet
                f2 = lambda x : 1 + r1 + thickness+yOffSet
                return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,xOffSet=xOffSet),n=nFunction)
            
            elif(r1 > 0 and r2 > 0):#meniscus concave case 1
                if(diameter == 0 or thickness == 0):
                    raise(Exception("Insufficient information provided for concave class object, both diameter and thickness must be positive"))
                rmax = r1
                f1 = lambda x : -(-(x-rmax-xOffSet)**2 + r1**2)**(1/2) + rmax+1+yOffSet
                f2 = lambda x : -(-(x-rmax-xOffSet)**2 + r2**2)**(1/2) + rmax+1 - thickness + r2 - r1+yOffSet
                if(diameter < 2*r2):#reasonable diameter
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,xOffSet=xOffSet),n=nFunction)
                else:#unreasonable diameter
                    raise(Exception("This is an unreasonable diameter for a meniscus concave lens"))
            elif(r1 < 0 and r2 < 0): #meniscus concave case 2
                r1, r2 = -r1, -r2
                if(diameter == 0 or thickness == 0):
                    raise(Exception("Insufficient information provided for concave class object, both diameter and thickness must be positive"))
                rmax = r2
                f1 = lambda x : (-(x-rmax-xOffSet)**2 + r1**2)**(1/2) +1 + yOffSet
                f2 = lambda x : (-(x-rmax-xOffSet)**2 + r2**2)**(1/2) + 1 - thickness - r2 + r1 + yOffSet
                if(diameter < 2*r1):#reasonable diameter
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,diameter,xOffSet=xOffSet),n=nFunction)
                else:#unreasonable diameter
                    raise(Exception("This is an unreasonable diameter for a meniscus concave lens"))
        else:
            raise(Exception("Not enough information supplied or diameter and thickness are 0 which is impossible"))#probably should print inputs
        
    #unsure how much burden I want to put on the user
    #we could take a dictionary of inputs which might be more readable but that might also make the whole process more complicated
    #insofar as translating that dictionary set up into our list structure
    @classmethod
    def custom(cls,surfaceList,nFunction=lambda x:1.5):
        return cls(surfaceList=surfaceList,n=nFunction)
    #there should probably some rhobust error checking for dealing with the defined surfaces
    


In [12]:
#[([xStart,xStop],[func1, func2]),...]

In [13]:
class driver:
    
    def __init__(self, n, listOfRays, listOfLenses=[], listOfMirrors=[]):
        self.nMedium = n
        self.listOfRays = listOfRays
        self.mapOfLenses = {i : listOfLenses[i] for i in range(len(listOfLenses))}
        self.listOfMirrors = listOfMirrors
        self.listOfLenses = listOfLenses
        self.surfaceList = []
        for i in range(len(listOfLenses)):
            self.surfaceList += listOfLenses[i].surfaceList
        for i in listOfMirrors:
            self.surfaceList += i.surfaceList
        for i in listOfRays:
            i.objIndex = self.locatingIndexFunction(i.sourceLocation)
                            
    def locatingIndexFunction(self, sourceLocation):
        """
        Overview: this function seeks to determine which index this location belongs to
        
        inputs: sourceLocation, (implicit list of lenses, list of mirrors)
        outputs: the label of the lenses in the map that contains this location or 
        -1 to indicate that it's in the surrounding medium
        
        Procedure:
        1. Iterate through the map of lenses and labels, choose a lens l_i
        2. Iterate through the list of surfaces associated with this lens, choose s_j
        ### ASSUMING NOT VERTICAL LINE ###
        3. if check s_j's domain bounds the x coordinate, if not proceed to next surface
        4. check if pairs of surfaces defined over the domain bound the y position by evaluating them
        ### VERTICAL LINE ####
        3. check if the line is equal within tolerance to the x coordinate
        4. check if the y position is bounded by the pairs of y coordinates that decribe the vertical line
        #### END OF CASES ####
        5. return if successful otherwise continue
        """
        x,y = sourceLocation
        mapOfLenses = self.mapOfLenses
        for label, lens in mapOfLenses.items():
            for orderedPair in lens.surfaceList:
                domain, surfaces = orderedPair
                if(len(domain) == 1):
                    if(len(surfaces) == 2):
                        if(x > domain[0]-0.001 and x < domain[0]+0.001 and y > surfaces[0] and y < surfaces[1]):
                            return label
                    else:
                        print("this is the pathelogical broken lens case")
                        raise(Exception("I haven't implemented this yet"))
                else:
                    if(x > domain[0] and x < domain[1]):#this could still be pathelogical 
                        for m in range(0,len(surfaces),2):
                            f1, f2 = surfaces[m], surfaces[m+1] #I assume that odd numbers of surfaces shouldn't be possible
                            lB, uB = sorted([f1(x),f2(x)])
                            if(lB < y and uB > y ):
                                return label
        return -1
                                
    def rayFunc(self,threshold=0,segments=8):
        """
        NOTE: 
        The beginning of this function is comprised of definitions of helper functions that the rayFunc needs to work
        each of these functions will be described under their definition.
        
        Inputs: threshold(Optional), segments(optional) (an instance of the class must call this method)
        Outputs: none
        
        ***
        These input/output lists are not really accurate as to what the function is doing since an instance of the object
        driver is modifying its own data members they don't need to be parameters for the function. A more accurate 
        description is, the driver is modifying the list of rays that was provided on construction of the instance
        by having those rays interact with the various optical elements provided in the list of lenses and list of mirrors
        given to the class initially. Hence the input and output are a list of rays.
        ***
        
        Description of procedure for rayFunc:
        1. Iterate through the list of rays, suppose we are on the ith step and have a ray r_i that we want to simulate
        2. Iterate through all surfaces provided to determine which the ray will impinge upon(if any).
            These surfaces are those which have a point that lies on the extension of the ray. The ray
            will interact with the surface that minimizes the Opitcal Path Length(OPL). Since we only work with
            homogenous media (that is, n is not a function of spatial coordinates within an element), the shortest
            path length will be a straight line. 
        3. Determine if the next medium is a lens, mirror, or the background by checking the position of the interaction
            point and searching for which element contains this point
        4. determine angle of intersection by using arccosine(r_i.unitVector dotproduct surfaceNormalVector)
        #### MIRROR ONLY ####
        5. Use law of reflection and a rotation matrix to specify the components of the reflected ray
        ->continue reading at the end of lens portion
        #### LENS ONLY ####
        5. Determine critical angle if n1 > n2
        6.a. If not total internal reflection:
            i. Use Snell's Law to calculate the angle relative to the normal of the refracted ray 
            ii. Use a rotation matrix to determine the components of the refracted ray
            iii. determine the intensity of the transmited (refracted ray) and, by conservation of energy, the intensity of
                the internally reflected ray as well
            iv. Calculate the s and p components of the reflected and transmitted ray
        6.b.  Use law of reflection and a rotation matrix to specify the components of the reflected ray
        #### END OF CASES ####
        7. update the ray object and record the normal vector
        
        """
        def normalVectorFunc(function, value): # Value is the x value where the given ray intersects the lens surface
            """
            This function returns the normal vector of a function at a specified point using a derivative based approach
            
            inputs: function, location
            outputs: an ordered pair corresponding to the xy components of the normal vector
            
            NOTE: The direction of the normal vector is ambiguous for the ray which requires a check later to ensure the
            correct orientation
            """
            
            fprime = float(mpmath.diff(function, value))
            return [-fprime/(1+fprime**2)**(0.5),1/(1+fprime**2)**(0.5)]
        
        def thetaCritical(n1, n2):#these should be effective indices in the case where they are functions
            """
            Determines theta critical for an interface
            input: 
            n1 is the index of refraction of the material the ray is currently in
            n2 is the index of refraction the next material
            output:
            critical angle (in rads)
            
            NOTE: the index of refraction for a medium varies by wavelength, so these have been represented as functions
            these functions must be evaluted prior to being passed into this function
            """
            return np.arcsin(n2/n1)
            #this is a value that cannot be reached and serves to tell the program that the ray won't internally reflect
        
        #expected inputs for line: [slope, x intercept], surface is a function, value is a reasonable estimate for where the zero occurs
        def rayIntersectSurface(line, surface, value): 
            """
            Given two functions, f1 and f2, f1 intersected f2 when f1(x) - f2(x) = 0
            This function is a wrapper for the fsolve routine in the scipy optimize suite
            
            inputs:
            line : a list containing the slope and x intersecpt of our ray
            surface : the surface that the ray might hit
            value: a guess to help fsolve find a root
            
            outputs:
            according to fsolve: a list of intersection points or the last value used if the search was unsuccessful
            """
            return opt.fsolve(func=lambda x: surface(x)-x*line[0] + line[0]*line[1], x0=value, xtol=1e-10, maxfev=200, factor=1)#f(x) - m(x-x0) = 0
        
        def refractedRayFunction(normalVector, angle, isNegative):
            """
            A function that calculates the components of the possible reflected or refracted ray depending on whether isNegative
            is true or false. If false, i.e. equal to 0 then the reflected ray is calculated. Otherwise the refracted ray
            is calculated. Either of these are achieved by rotating the normal vector.
            
            NOTE:
            The rotation matrix method is ambigious about whether the angle that we are rotating about is positive or negative.
            Without a sure way to calculate the correct one only, we calculate both possibilities and take the one that has the greater 
            dot product to the ray's unit vector
            
            input:
            normalVector, angle to rotate by, isNegative - a flag to indicate whether we are using the normal vector or 
            minus the normal vector
            
            output:
            two lists containing the components of the rotated rays
            """
            xN, yN = normalVector
            if(isNegative):
                xN, yN = -xN, -yN
            refractedRay1 = [xN*np.cos(-theta2) - yN*np.sin(-theta2), xN*np.sin(-theta2) + yN*np.cos(-theta2)]
            refractedRay2 = [xN*np.cos(theta2) - yN*np.sin(theta2), xN*np.sin(theta2) + yN*np.cos(theta2)]
            return refractedRay1, refractedRay2
        #lets make it more modular :D
        def surfaceFinderPart(ray, listOfElements,segments):
            """
            Inputs: 
            ray : a ray that we are seeking to have intersect a surface
            listOfElements: a list containing either lenses or mirrors that we are going to search their list of surfaces
            
            Outputs:
            xMin, yMin represent the (x,y) of the nearest location of intersection
            minDistSurface is the function that minimized the OPL for this set of optical elements
            minDistance is the length of the OPL
            
            Procedure: (for not vertical lines)
            1. iterate through the list of elements, select one say e_i
            2. iterate through its list of surfaces, s_j
            ### FOR NOT VERTICAL LINES ###
            3. compare the domain of the surface to the point's x location
            4.a. if the x location is not bounded by the domain iterate to next surface
            4.b  if the x location is bounded, break the domain into 8 (or more if desired) evenly spaced intervals
            5. for each interval check if the rayIntersectSurface function returns a valid point (that is a point which is along 
                the extension of the ray)
            ### FOR VERTICAL LINES ### !If there are multiple vertical lines which share a x location, this should iterate through each line segment
            3. check if x location of the point is within tolerance equal to the vertical line, if not iterate to next lens
            4. check if y location is bounded by the line segment, if not iterate to next lens
            ### END OF CASES ###
            6. if a valid point is found, compare its OPL to the best OPL so far, if better, update the quantities   
            """
            x0,y0 = ray.sourceLocation
            minDistance = 1e10
            minDistSurface = -1
            xMin, yMin = -1,-1
            for element in listOfElements:
                listOfSurfaces = element.surfaceList
                for s in range(len(listOfSurfaces)):
                    domain, surfaces = listOfSurfaces[s]
                    #should add the shifted case, where there are two line segments 
                    if(len(domain) == 1):#this is a vertical line 
                        pointOfIntersection = ray.slope*domain[0] + ray.yIntercept
                        if(pointOfIntersection >= surfaces[0] and pointOfIntersection <= surfaces[1]):
                            distance = ((domain[0]-x0)**2 + (pointOfIntersection-y0)**2)**(1/2)
                            checkAngle = np.arctan2(pointOfIntersection-y0,domain[0]-x0)
                            rayAngle = ray.angle
                            if(checkAngle < 0):
                                checkAngle += 2*pi
                            if(checkAngle > rayAngle+0.001 or checkAngle < rayAngle-0.001): #this ray intersection is clearly bad so disregard it
                                continue
                            if(distance < minDistance and distance > 0.001):
                                minDistance = distance
                                xMin, yMin = domain, pointOfIntersection
                                minDistSurface = [domain[0],surfaces[:2]] #I use this as the label to differentiate between the vertical line case and the rest
                        elif(len(surfaces) == 4):
                            print("update this part with the pieces from above ")
                            if(pointOfIntersection >= surfaces[2] and pointOfIntersection <= surfaces[3]):
                                distance = ((domain[0]-x0)**2 + (pointOfIntersection-y0)**2)**(1/2)
                                if(distance < minDistance and distance > 0.001):
                                    minDistance = distance
                                    xMin, yMin = domain, pointOfIntersection
                                    minDistSurface = [domain[0],surfaces[2:4]]
                    #should probably include error checking to ensure that the length of any of these domain lists is either 1 or 2 
                    else:#we're not working with a vertical line
                        #given a domain (a,b), my strategy is to use fsolve with an intial guess of (a+b)/2, if the ray happens to already be on the
                        #surface, we refine the search by splitting the surface into eighths
                        a, b = domain
                        initial_guess = [i*(a+b)/segments for i in range(segments)] #somewhat arbitarily chose to split the domain into eighths 
                        for guess in initial_guess:
                            for i in range(len(surfaces)):#we have to loop over all the surfaces defined on this domain
                                surface_i = surfaces[i]
                                listOfPointsOfIntersection = rayIntersectSurface([ray.slope, ray.xIntercept], surface_i, guess)
                                for pointOfIntersection in listOfPointsOfIntersection:
                                    if(pointOfIntersection > a and pointOfIntersection < b):#we've got a valid intersection
                                        y1 = surface_i(pointOfIntersection)
                                        distance = ((pointOfIntersection-x0)**2 + (y1-y0)**2)**(1/2)
                                        checkAngle = np.arctan2(y1-y0,pointOfIntersection-x0)
                                        rayAngle = ray.angle
                                        if(checkAngle < 0):
                                            checkAngle+=2*pi
                                        if(checkAngle > rayAngle+0.001 or checkAngle < rayAngle-0.001): #this ray intersection is clearly bad so disregard it
                                            continue
                                        if(distance < minDistance and distance > 0.001):#put in a min distance, this might fix the ray not moving problem
                                            minDistance = distance
                                            xMin, yMin = pointOfIntersection, y1
                                            minDistSurface = [surface_i]
            return xMin, yMin, minDistSurface, minDistance
        
        segments = int(segments)
        if(segments <= 1):
            segments = 2
        passes = 0
        listOfRays = self.listOfRays
        while(passes < 12):
            listOfThingsToAdd = []
            for i in range(len(listOfRays)):
                ray = listOfRays[i]
                if(ray.intensity < threshold or not ray.worthUsing):
                    continue
                isMirror = 0
                xMin1,yMin1, minDistSurface1, distance1 = surfaceFinderPart(ray,self.listOfLenses,segments=segments)
                xMin2,yMin2, minDistSurface2, distance2 = surfaceFinderPart(ray,self.listOfMirrors,segments=segments)
                if(distance1 > 0 and distance2 > 0):
                    if(distance1 < distance2):
                        xMin, yMin, minDistSurface = xMin1, yMin1, minDistSurface1
                    else:
                        isMirror = 1
                        xMin, yMin, minDistSurface = xMin2, yMin2, minDistSurface2
                elif(distance1 > 0):
                    xMin, yMin, minDistSurface = xMin1, yMin1, minDistSurface1
                elif(distance2 > 0):
                    isMirror = 1
                    xMin, yMin, minDistSurface = xMin2, yMin2, minDistSurface2
                else:#no points of intersection, may as well move on to the next ray
                    ray.worthUsing = 0
                    continue
                #we have now found the next point of intersection or exhausted all possibilities so we can move on to the rest of this
                #before we proceed we again need to have the special case handled for vertical lines
                if(minDistSurface == -1):
                    ray.worthUsing = 0
                    continue
                if(len(minDistSurface) == 2): #the special case
                    xMin, normalVector = xMin[0], [-1,0]
                else:
                    normalVector = normalVectorFunc(minDistSurface[0],xMin)
                if(np.dot(normalVector, ray.unitVector) > 0):
                    normalVector = [-i for i in normalVector]
                theta1 = np.arccos(np.dot(ray.unitVector, normalVector))
                
                if(not isMirror):
                    if(len(minDistSurface)==2):
                        nextObj1 = self.locatingIndexFunction((xMin+0.001, yMin))
                        nextObj2 = self.locatingIndexFunction((xMin-0.001, yMin))
                    else:
                        nextObj1 = self.locatingIndexFunction((xMin, yMin+0.0001))
                        nextObj2 = self.locatingIndexFunction((xMin, yMin-0.0001))
                    if(nextObj1 != ray.objIndex):
                        nextObj = nextObj1
                    else:
                        nextObj = nextObj2

                    if(ray.objIndex == -1 and nextObj == -1):
                        continue
                    elif(ray.objIndex == -1):
                        n1 = self.nMedium
                        n2 = self.mapOfLenses.get(nextObj).refractiveIndex
                    elif(nextObj == -1):
                        n1 = self.mapOfLenses.get(ray.objIndex).refractiveIndex
                        n2 = self.nMedium
                    else:
                        n1 = self.mapOfLenses.get(ray.objIndex).refractiveIndex
                        n2 = self.mapOfLenses.get(nextObj).refractiveIndex    
                    wavelength = ray.wavelength
                    if(theta1 > pi/2):
                        theta1 = pi - theta1
                    if(abs(n1(wavelength)) > abs(n2(wavelength)) and theta1 >= thetaCritical(n1(wavelength),n2(wavelength))):#internal reflection case need to do this still
                        theta2 = theta1
                        refractedRay1, refractedRay2 = refractedRayFunction(normalVector, theta2, 0)
                        if(np.dot(refractedRay1, ray.unitVector) > np.dot(refractedRay2, ray.unitVector)):#don't use absolute values here
                            refractedRay = refractedRay1
                        else:
                            refractedRay = refractedRay2
                        ray.intensityList.append(ray.intensity)
                        nextObj = ray.objIndex
                    else:
                        i0 = ray.intensity
                        if(theta1 == 0):
                            theta2 = 0
                            refractedRay1, refractedRay2 = refractedRayFunction(normalVector, theta2, 1)
                            if(abs(np.dot(refractedRay1, ray.unitVector)) > abs(np.dot(refractedRay2, ray.unitVector))):
                                refractedRay = refractedRay1
                            else:
                                refractedRay = refractedRay2
                            Tperpendicular =  n2(wavelength)/n1(wavelength)*(2*n1(wavelength)/(n1(wavelength)+n2(wavelength)))**2
                            Tparallel = n2(wavelength)/n1(wavelength)*(2*n1(wavelength)/(n1(wavelength)+n2(wavelength)))**2
                            ray.perpendiular = Tperpendicular/(Tperpendicular+Tparallel)
                            ray.parallel = Tparallel/(Tperpendicular+Tparallel)
                            ray.update_intensity_history(theta1, theta2, n1, n2)
                            reflectedXaxisTheta = ray.angle
                            newRay = Ray(xMin, yMin, reflectedXaxisTheta, intensity = abs(i0 - ray.intensity))
                            Rperp = (n2(wavelength)-n1(wavelength)/(n2(wavelength)+n1(wavelength)))**2
                            Rpara = (n2(wavelength)-n1(wavelength)/(n2(wavelength)+n1(wavelength)))**2
                            newRay.perpendicular = Rperp/(Rperp+Rpara)
                            newRay.parallel = Rpara/(Rperp+Rpara)
                            newRay.objIndex = ray.objIndex
                        else:
                            theta2 = np.arcsin((n1(wavelength)/n2(wavelength))*np.sin(theta1))
                            refractedRay1, refractedRay2 = refractedRayFunction(normalVector, theta2, 1)
                            if(abs(np.dot(refractedRay1, ray.unitVector)) > abs(np.dot(refractedRay2, ray.unitVector))):
                                refractedRay = refractedRay1
                            else:
                                refractedRay = refractedRay2
                            reflect1, reflect2 = refractedRayFunction(normalVector, theta1, 0)
                            if(np.dot(reflect1, ray.unitVector) > np.dot(reflect2, ray.unitVector)):#don't use absolute values here
                                reflect = reflect1
                            else:
                                reflect = reflect2
                            Tperpendicular =  n2(wavelength)*np.cos(theta2)/(n1(wavelength)*np.cos(theta1))*(Ray.tPerp(theta1,theta2, n1,n2,wavelength))**2
                            Tparallel = n2(wavelength)*np.cos(theta2)/(n1(wavelength)*np.cos(theta1))*(Ray.tPara(theta1,theta2,n1,n2,wavelength))**2
                            ray.perpendiular = Tperpendicular/(Tperpendicular+Tparallel)
                            ray.parallel = Tparallel/(Tperpendicular+Tparallel)
                            ray.update_intensity_history(theta1, theta2, n1, n2)
                            reflectedXaxisTheta = np.arctan2(reflect[1],reflect[0])
                            if(reflectedXaxisTheta < 0):
                                reflectedXaxisTheta += 2*np.pi
                            newRay = Ray(xMin, yMin, reflectedXaxisTheta, intensity = (abs(i0 - ray.intensity)))
                            Rperp = Ray.rPerp(theta1,theta2,n1,n2,wavelength)**2
                            Rpara = Ray.rPara(theta1,theta2,n1,n2,wavelength)**2
                            newRay.perpendicular = Rperp/(Rperp+Rpara)
                            newRay.parallel = Rpara/(Rperp+Rpara)
                            newRay.objIndex = ray.objIndex
                            #comment this part out to look at the other behavior
                            listOfThingsToAdd.append(newRay)
                else:
                    #we can add the loss function bit later
                    theta2 = theta1
                    refractedRay1, refractedRay2 = refractedRayFunction(normalVector, theta2, 1)
                    if(np.dot(refractedRay1, ray.unitVector) > np.dot(refractedRay2, ray.unitVector)):#don't use absolute values here
                        refractedRay = refractedRay1
                    else:
                        refractedRay = refractedRay2
                    ray.intensityList.append(ray.intensity)
                    nextObj = ray.objIndex
                
                ray.normalVectorHistory.append([[xMin, normalVector[0]+xMin],[yMin,normalVector[1]+yMin]])
                refractedXaxisTheta = float(mpmath.atan2(mpmath.mpmathify(refractedRay[1]),mpmath.mpmathify(refractedRay[0])))
                if(refractedXaxisTheta < 0):
                    refractedXaxisTheta += 2*np.pi
                ray.angleHistory.append(theta1*180/pi)
                ray.angleHistory.append(theta2*180/pi)
                ray.update_location_history(xMin,yMin,refractedXaxisTheta,nextObj)
                    #uncomment this part to keep working on the fresnel stuff
            listOfRays+=listOfThingsToAdd
            passes+=1
    def plotter(self,fig): #this method will probably keep growing however it's not too bad right now
        sub = fig.add_subplot(111)
        surfaceList = self.surfaceList
        #first we have to handle graphing the lens or lenses
        for s in range(len(surfaceList)):
            domain, surfaces = surfaceList[s]
            if(len(domain) == 1):#this will need some refining for the 4 point case
                x = domain[0]
                xList = [x]*len(surfaces)
                sub.plot(xList,surfaces)
            else:
                x = np.linspace(domain[0],domain[1],1000)
                for function in surfaces:#this will probably need special handling for constant functions
                    tester = function(x[:20])
                    if(type(tester) is int or type(tester) is float):
                        yList = [function(domain[0])]*len(x)
                    else:
                        yList = function(x)
                    sub.plot(x, yList)
        rayList = self.listOfRays
        for i in range(len(rayList)):
            xAx, yAx = zip(*rayList[i].locationHistory)
            xAxList, yAxList = list(xAx), list(yAx)
            length = len(xAx)
            ray = rayList[i]
            angle = ray.angle
            slope = ray.slope
            ray1, ray2 = [xAxList[-1]-5, yAxList[-1]-5*slope],[xAxList[-1]+5, yAxList[-1]+5*slope]
            checkAngle = np.arctan2(ray1[1]-yAxList[-1],ray1[0]-xAxList[-1])
            if(checkAngle < 0):
                checkAngle+=2*pi
            if(checkAngle > angle +0.001 or checkAngle < angle - 0.001):
                xAxList.append(ray2[0])
                yAxList.append(ray2[1])
            else:
                xAxList.append(ray1[0])
                yAxList.append(ray1[1])
                
            for j in range(len(ray.intensityList)):#change this piece for different plot configurations
                #if(rayList[i].intensityList[j] < 0.5):
                #    continue
                #sub.plot(xAxList[j:j+2],yAxList[j:j+2],color=plt.cm.viridis(1-rayList[i].intensityList[j]))#,alpha=(rayList[i].intensityList[j]))
                #sub.plot(xAxList[j:j+2],yAxList[j:j+2],color=plt.cm.gray(1-rayList[i].intensityList[j]))
                sub.plot(xAxList[j:j+2],yAxList[j:j+2],'k',alpha=(ray.intensityList[j]))#,label="ray {0}".format(ray.identity))
            
            #sub.plot(xAxList, yAxList, 'b',color=plt.cm.jet(i/len(rayList)), label="ray {0}".format(rayList[i].identity))
            #for normalVector in ray.normalVectorHistory:
            #    sub.plot(normalVector[0],normalVector[1],'m-.')
            
    def plotterJustLens(self,fig): #for debugging
        sub = fig.add_subplot(111)
        surfaceList = self.surfaceList
        for s in range(len(surfaceList)):
            domain, surfaces = surfaceList[s]
            if(len(domain) == 1):#this will need some refining for the 4 point case
                x = domain[0]
                xList = [x]*len(surfaces)
                sub.plot(xList,surfaces)
            else:
                x = np.linspace(domain[0],domain[1],1000)
                for function in surfaces:#this will probably need special handling for constant functions
                    if(function(1) == function(2) and function(1) == function(1.5)):#this seems to be a constant function, should probably put in more 
                        #robust handling when the domain for the contain function isn't so nice
                        yList = [function(1)]*len(x)
                    else:
                        yList = function(x)
                    sub.plot(x, yList)

bug list:
1. problem with getting the surface finding feature to "jump" to the next surface, i.e. not just have the ray stand still
2. problem with refracting off of the flat sides in a concave lens
3. there probably should be explicit handling for vertical incoming rays
4. the meniscus concave predefined doesn't seem to work, unsure why
    removing the passes patch seemed to make some progress
5. gaussian optics also seem to need some work
6. width isn't working properly for the meniscus concave part
7. there is no change in the ray propagation going through a flat vertical surface
8. some new bugs have emerged since I have been working on the internal reflection portion
9. moving meniscus convex 5 to left causes the plots to stop working

to do:
1. should minimize domain such that the smaller of the two curves is the limiting factor
2. the biconvex and I suspect the other lenses too need some help with rays that would internally reflect
3. more testing with the beam source version
4. the slope from the right flag needs to be updated when we reflect off of a horizontal surface
5. check if the assumption that we have to be in the 1st quandrant is the cause of the failure of the program
6. add the ability for convex sections that are normally not intersecting
7. deal with the gaussian optics case where both r1, r2 are negative
8. work on the guess procedure for the surface intersection
9. add the ability for a ray to hit the same surface twice (or more)

fixed:(big)
1. bug with the fringe rays intersecting in the middle

In [14]:
#fix problem with updating objIndex
#bug with plano concave r1 = 0, r2 < 0
#bug at -1.5 for l2, it's because our rays were generated in the l2 lens
#should improve efficiency of ray propagating algorithm by using a flag to indicate whether a ray is worth working on
#

Ray.rayCount = 0

myRayList = RayGenerator.beam_source(4,-3,6,-3,11)

#myLens = Lens.gaussianOptics(r1=3, r2=0,diameter=3, thickness=0.5,xOffSet=2,yOffSet=2,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = 0, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = 1.9, nFunction = lambda x: 1.5)
#myLens2 = Lens.predefined("biconcave", nFunction=lambda x : 2, xOffSet=0,yOffSet=2)

#myLens = Lens.gaussianOptics(r1=5, r2=-5,diameter=3, thickness=0.5,xOffSet=0,yOffSet=-1,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = -3, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = -4.75, nFunction = lambda x: 1.5)
#myMirror = Mirror.custom(surfaceList =[[[0,10],[lambda x : -(x-5)**2+8]]])
#myMirror = Mirror.custom(surfaceList = [[[0,10],[lambda x: -((x-5)**2+1)**(1/2)+8]]])
f1 = lambda x : -((x-5)**2+1)**(1/2)+8
f2 = lambda x : 1

l=[Lens.custom(surfaceList=[[[3],[f2(3), f1(3)]],[[3,7],[f2,f1]],[[7],[f2(7),f1(7)]]])]
m=[]
theDriver = driver(lambda x:1,myRayList.theRayList, listOfLenses=l,listOfMirrors=m)
theDriver.rayFunc(threshold=0)
fig = plt.figure()

theDriver.plotter(fig)
plt.ylim(-3,14)
plt.xlim(1,9)
plt.legend()
plt.tight_layout()
plt.show()

In [20]:
#fix problem with updating objIndex
#bug with plano concave r1 = 0, r2 < 0
#bug at -1.5 for l2, it's because our rays were generated in the l2 lens
#should improve efficiency of ray propagating algorithm by using a flag to indicate whether a ray is worth working on
#

Ray.rayCount = 0

myRayList = RayGenerator.beam_source(4,-3,6,-3,11)

#myLens = Lens.gaussianOptics(r1=3, r2=0,diameter=3, thickness=0.5,xOffSet=2,yOffSet=2,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = 0, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = 1.9, nFunction = lambda x: 1.5)
#myLens2 = Lens.predefined("biconcave", nFunction=lambda x : 2, xOffSet=0,yOffSet=2)

myLens = Lens.gaussianOptics(r1=5, r2=-5,diameter=3, thickness=0.5,xOffSet=0,yOffSet=-1,nFunction=lambda x: 1.5)
myLens2 = Lens.gaussianOptics(r1 = -3, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = -4.75, nFunction = lambda x: 1.5)
myMirror = Mirror.custom(surfaceList =[[[0,10],[lambda x : -(x-5)**2+8]]])
#myMirror = Mirror.custom(surfaceList = [[[0,10],[lambda x: -((x-5)**2+1)**(1/2)+8]]])
#f1 = lambda x : -((x-5)**2+1)**(1/2)+8
#f2 = lambda x : 1
l=[myLens, myLens2]
m=[myMirror]
#l=[Lens.custom(surfaceList=[[[3],[f2(3), f1(3)]],[[3,7],[f2,f1]],[[7],[f2(7),f1(7)]]])]
#m=[]
theDriver = driver(lambda x:1,myRayList.theRayList, listOfLenses=l,listOfMirrors=m)
theDriver.rayFunc(threshold=1e-3)
fig = plt.figure()

theDriver.plotter(fig)
plt.ylim(-3,14)
plt.xlim(1,9)
plt.legend()
plt.tight_layout()
plt.show()

In [16]:
#plano hyperbola example, eccentricity = nl/nm = (2)**(1/2)
#fix problem with updating objIndex
#bug with plano concave r1 = 0, r2 < 0
#bug at -1.5 for l2, it's because our rays were generated in the l2 lens
#should improve efficiency of ray propagating algorithm by using a flag to indicate whether a ray is worth working on
#

Ray.rayCount = 0

myRayList = RayGenerator.point_source(5,8+2**0.5,11,75*pi/180+pi,105*pi/180+pi)

#myLens = Lens.gaussianOptics(r1=3, r2=0,diameter=3, thickness=0.5,xOffSet=2,yOffSet=2,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = 0, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = 1.9, nFunction = lambda x: 1.5)
#myLens2 = Lens.predefined("biconcave", nFunction=lambda x : 2, xOffSet=0,yOffSet=2)

#myLens = Lens.gaussianOptics(r1=5, r2=-5,diameter=3, thickness=0.5,xOffSet=0,yOffSet=-1,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = -3, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = -4.75, nFunction = lambda x: 1.5)
#myMirror = Mirror.custom(surfaceList =[[[0,10],[lambda x : -(x-5)**2+8]]])
#myMirror = Mirror.custom(surfaceList = [[[0,10],[lambda x: -((x-5)**2+1)**(1/2)+8]]])
f1 = lambda x : -((x-5)**2+1)**(1/2)+8
f2 = lambda x : 1

l=[Lens.custom(surfaceList=[[[1],[f2(1), f1(1)]],[[1,9],[f2,f1]],[[9],[f2(9),f1(9)]]],nFunction = lambda x : 2**0.5)]
m=[]
theDriver = driver(lambda x:1,myRayList.theRayList, listOfLenses=l,listOfMirrors=m)
theDriver.rayFunc(threshold=0)
fig = plt.figure()

theDriver.plotter(fig)
plt.ylim(-3,20)
plt.xlim(1,9)
plt.legend()
plt.tight_layout()
plt.show()

In [21]:
#biconvex hyperbola example, eccentricity = nl/nm = (2)**(1/2)

#eccentricity for hyberbola = (1+(b/a)**2)**(1/2)
#fix problem with updating objIndex
#bug with plano concave r1 = 0, r2 < 0
#bug at -1.5 for l2, it's because our rays were generated in the l2 lens
#should improve efficiency of ray propagating algorithm by using a flag to indicate whether a ray is worth working on
#

Ray.rayCount = 0

myRayList = RayGenerator.point_source(5,3,11,75*pi/180+pi,105*pi/180+pi)

#myLens = Lens.gaussianOptics(r1=3, r2=0,diameter=3, thickness=0.5,xOffSet=2,yOffSet=2,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = 0, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = 1.9, nFunction = lambda x: 1.5)
#myLens2 = Lens.predefined("biconcave", nFunction=lambda x : 2, xOffSet=0,yOffSet=2)

#myLens = Lens.gaussianOptics(r1=5, r2=-5,diameter=3, thickness=0.5,xOffSet=0,yOffSet=-1,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = -3, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = -4.75, nFunction = lambda x: 1.5)
#myMirror = Mirror.custom(surfaceList =[[[0,10],[lambda x : -(x-5)**2+8]]])
#myMirror = Mirror.custom(surfaceList = [[[0,10],[lambda x: -((x-5)**2+1)**(1/2)+8]]])
f1 = lambda x : -(4/5*(x-5)**2+4)**(1/2)+0
f2 = lambda x : (4/5*(x-5)**2+4)**(1/2)-9

l=[Lens.custom(surfaceList=[[[1],[f2(1), f1(1)]],[[1,9],[f2,f1]],[[9],[f2(9),f1(9)]]],nFunction = lambda x : 1.5)]
m=[]
theDriver = driver(lambda x:1,myRayList.theRayList, listOfLenses=l,listOfMirrors=m)
theDriver.rayFunc(threshold=0)
fig = plt.figure()

theDriver.plotter(fig)
plt.ylim(-13,4)
plt.xlim(0,10)
plt.legend()
plt.tight_layout()
plt.show()

In [17]:
#biconvex elliptic example, eccentricity = nl/nm = (2)**(1/2)

#eccentricity for hyberbola = (1+(b/a)**2)**(1/2)
#fix problem with updating objIndex
#bug with plano concave r1 = 0, r2 < 0
#bug at -1.5 for l2, it's because our rays were generated in the l2 lens
#should improve efficiency of ray propagating algorithm by using a flag to indicate whether a ray is worth working on
#

Ray.rayCount = 0

myRayList = RayGenerator.point_source(2+5,0,11,-30*pi/180+pi,30*pi/180+pi)

#myLens = Lens.gaussianOptics(r1=3, r2=0,diameter=3, thickness=0.5,xOffSet=2,yOffSet=2,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = 0, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = 1.9, nFunction = lambda x: 1.5)
#myLens2 = Lens.predefined("biconcave", nFunction=lambda x : 2, xOffSet=0,yOffSet=2)

#myLens = Lens.gaussianOptics(r1=5, r2=-5,diameter=3, thickness=0.5,xOffSet=0,yOffSet=-1,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = -3, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = -4.75, nFunction = lambda x: 1.5)
#myMirror = Mirror.custom(surfaceList =[[[0,10],[lambda x : -(x-5)**2+8]]])
#myMirror = Mirror.custom(surfaceList = [[[0,10],[lambda x: -((x-5)**2+1)**(1/2)+8]]])
f1 = lambda x : -(-5/9*(x-5)**2+5)**(1/2)
f2 = lambda x : (-5/9*(x-5)**2+5)**(1/2)

l=[Lens.custom(surfaceList=[[[2],[f2(2), f1(2)]],[[2,8],[f2,f1]],[[8],[f2(8),f1(8)]]],nFunction = lambda x : 1.5)]
m=[]
theDriver = driver(lambda x:1,myRayList.theRayList, listOfLenses=l,listOfMirrors=m)
theDriver.rayFunc(threshold =1e-3,segments=32)
fig = plt.figure()

theDriver.plotter(fig)
plt.ylim(-5,5)
plt.xlim(0,10)
plt.legend()
plt.tight_layout()
plt.show()

In [18]:
#

Ray.rayCount = 0

myRayList = RayGenerator.point_source(2+5,0,11,-30*pi/180+pi,30*pi/180+pi)

#myLens = Lens.gaussianOptics(r1=3, r2=0,diameter=3, thickness=0.5,xOffSet=2,yOffSet=2,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = 0, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = 1.9, nFunction = lambda x: 1.5)
#myLens2 = Lens.predefined("biconcave", nFunction=lambda x : 2, xOffSet=0,yOffSet=2)

#myLens = Lens.gaussianOptics(r1=5, r2=-5,diameter=3, thickness=0.5,xOffSet=0,yOffSet=-1,nFunction=lambda x: 1.5)
#myLens2 = Lens.gaussianOptics(r1 = -3, r2 = 3, diameter = 3, thickness = 0.5, xOffSet = 2, yOffSet = -4.75, nFunction = lambda x: 1.5)
#myMirror = Mirror.custom(surfaceList =[[[0,10],[lambda x : -(x-5)**2+8]]])
#myMirror = Mirror.custom(surfaceList = [[[0,10],[lambda x: -((x-5)**2+1)**(1/2)+8]]])
#myLens = Lens.gaussianOptics(r1 = 5, r2 = -5, diameter = 0, thickness=10, xOffSet=0, yOffSet=0, nFunction=lambda x : 1.5)


f1 = lambda x : (-(x-5-xOffSet)**2 + 9)**(1/2) + 1 +yOffSet
f2 = lambda x : -(-(x-5-xOffSet)**2 + 9)**(1/2) + 8 + yOffSet
l=[myLens]
m=[]
theDriver = driver(lambda x:1,myRayList.theRayList, listOfLenses=l,listOfMirrors=m)
#theDriver.plotterJustLens(threshold =1e-3,segments=32)
fig = plt.figure()

theDriver.plotterJustLens(fig)
plt.ylim(-5,5)
plt.xlim(0,10)
plt.legend()
plt.tight_layout()
plt.show()

In [710]:
#print(ray.unitVectorHistory)

In [ ]:
def sellmeier(coeffList,wavelength):
    B1,B2,B3,C1,C2,C3 = coeffList
    if(B1 and B2 and B3 and C1 and C2 and C3):
        wavelength *= 1e6
        return lambda x : (1 + B1*wavelength**2/(wavelength**2 - C1) + B2*wavelength**2/(wavelength**2 - C2) + B3*wavelength**2/(wavelength**2 - C3) )**(1/2)
    return refractiveIndex